In [33]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Load labeled landmarks and corresponding labels
# df = pd.read_csv('data/processed/labeled_hand_landmarks.csv')
# df = pd.read_csv('E:\Projects\Sign Language Project\SignSpeak\data\processed\labeled_hand_landmarks.csv')
df = pd.read_csv('E:\Projects\Sign Language Project\SignSpeak\data\processed\simple_signs\labeled_hand_landmarks.csv')

# Number of frames per sequence
NUM_FRAMES = 30  # Adjust based on your video data

# Reshape the data into sequences of frames
def create_sequences(data, labels, num_frames=NUM_FRAMES):
    sequences, sequence_labels = [], []
    for i in range(len(data) - num_frames):
        # Take a sequence of num_frames
        sequence = data[i:i+num_frames]
        sequences.append(sequence)
        # Use the label of the last frame in the sequence
        sequence_labels.append(labels[i + num_frames - 1])
    return np.array(sequences), np.array(sequence_labels)

# Drop the label column
X = df.drop('label', axis=1).values
y = df['label'].values

# Reshape the data into sequences of frames
X_seq, y_seq = create_sequences(X, y, NUM_FRAMES)

# Encode labels
label_encoder = LabelEncoder()
y_seq_encoded = label_encoder.fit_transform(y_seq)
y_seq_encoded = to_categorical(y_seq_encoded)

# Split into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X_seq, y_seq_encoded, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f"Training samples: {X_train.shape[0]}")
print(f"Validation samples: {X_val.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")

# Verify the shape of X_train and X_val
print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_val: {X_val.shape}")

# Reshape X_train and X_val if necessary
# X_train = X_train.reshape((-1, 30, 63))
# X_val = X_val.reshape((-1, 30, 63))

Training samples: 63
Validation samples: 13
Testing samples: 14
Shape of X_train: (63, 30, 21)
Shape of X_val: (13, 30, 21)


In [34]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Define LSTM model
def build_lstm_model(input_shape, num_classes):
    model = Sequential()

    # LSTM layers
    model.add(LSTM(64, return_sequences=True, input_shape=input_shape))
    model.add(Dropout(0.2))

    model.add(LSTM(64))
    model.add(Dropout(0.2))

    # Fully connected layer
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.2))

    # Output layer
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Define input shape
input_shape = (NUM_FRAMES, 63)  # 30 frames, 63 keypoints per frame
num_classes = y_train.shape[1]

# Build the model
model = build_lstm_model(input_shape, num_classes)

# Model summary
model.summary()


Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_20 (LSTM)              (None, 30, 64)            32768     
                                                                 
 dropout_30 (Dropout)        (None, 30, 64)            0         
                                                                 
 lstm_21 (LSTM)              (None, 64)                33024     
                                                                 
 dropout_31 (Dropout)        (None, 64)                0         
                                                                 
 dense_20 (Dense)            (None, 64)                4160      
                                                                 
 dropout_32 (Dropout)        (None, 64)                0         
                                                                 
 dense_21 (Dense)            (None, 4)               

In [35]:
# Check if the total number of elements is divisible by 30 * 63
if X_train.size % (30 * 63) == 0 and X_val.size % (30 * 63) == 0:
    # Reshape X_train and X_val if necessary
    X_train = X_train.reshape((-1, 30, 63))
    X_val = X_val.reshape((-1, 30, 63))
else:
    raise ValueError(
        "The total number of elements in X_train or X_val is not divisible by 30 * 63")

# Set training parameters

batch_size = 32
epochs = 30


# Train the model

history = model.fit(X_train, y_train, validation_data=(
    X_val, y_val), epochs=epochs, batch_size=batch_size)




# Save the model after training


model.save('models/landmark models/sign_language_lstm.h5')

ValueError: The total number of elements in X_train or X_val is not divisible by 30 * 63

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(
    X_test, y_test, batch_size=batch_size)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()